In [ ]:
import time
import asyncio

In [ ]:
async def count():
    print("One")
    await asyncio.sleep(1)
    print("Two")

async def main():
    await asyncio.gather(count(), count(), count())

s = time.perf_counter()

# try:
#     asyncio.run(main())
# except RuntimeError as e:
#     await main()

await main()

elapsed = time.perf_counter() - s
print(f"executed in {elapsed:0.2f} seconds.")
    

In [ ]:
dir(asyncio)

In [ ]:
loop = asyncio.get_running_loop()

In [ ]:
asyncio.get_event_loop_policy()._local._loop.is_running()

In [ ]:
dir(asyncio.get_event_loop_policy())